In [1]:
import numpy as np

In [2]:
dnames = [
    "data/data_TOHLC_TH_2022.08.26.txt",
    "data/data_TOHLC_TH_2022.08.29.txt",
    "data/data_TOHLC_TH_2022.08.30.txt",
    "data/data_TOHLC_TH_2022.08.31.txt",
    "data/data_TOHLC_TH_2022.09.07.txt",
    "data/data_TOHLC_TH_2022.09.08.txt",
    "data/data_TOHLC_TH_2022.09.09.txt",
    "data/data_TOHLC_TH_2022.09.10.txt",
]

def loaddata(dname,npoints,ydist):
    """
    dname   = location of data file
    npoints = nr. points used as a row to train on
    ydist   = nr. timesteps away to set target
    """
    
    # load raw data
    data_raw = np.genfromtxt(dname,delimiter=",")
    # drop first column (date)
    data_noT = data_raw[:,1:]
    
    # combine data to make input rows
    data_npoints = np.array([])
    for i in range(len(data_noT)-npoints):
        newrow = data_noT[i]
        for j in range(1,npoints):
            k = i+j
            newrow = np.concatenate([newrow,data_noT[k]])
        if i==0:
            data_npoints = newrow
        else:
            data_npoints = np.vstack((data_npoints,newrow))

    # targets start at row ydist, take last element (close)
    targets = data_npoints[ydist:,-1]
    
    # final ydist rows of data_npoints don't have targets
    # -- delete them
    data_ysize = data_npoints[:-ydist]
    
    # find opening values for each row in input
    openvals = data_ysize[:,0]
    # go from (XXX,) to (XXX,1) for broadcasting
    openvals = openvals[:,np.newaxis] 
    targets  = targets[:,np.newaxis]
    
    # subtract opening value from row in input/targets
    data_sub = data_ysize - openvals
    target_sub = targets - openvals

    return data_sub, target_sub, openvals    

In [3]:
nb = 20  # number of bars to include in lookback
yd = 3  # number of steps past input to set target
#for dn in dnames:
#    loaddata(dn, nb)
#dn = "data/data_TOHLC_TH_2022.08.29.txt"

In [4]:
X_total = np.array([])
y_total = np.array([])
openvals_total = np.array([])
i=0
for dn in dnames:
    X_tmp, y_tmp, openvals_tmp = loaddata(dn,nb,yd)
    if i==0:
        X_total = X_tmp
        y_total = y_tmp
        openvals_total = openvals_tmp
    else:
        X_total = np.vstack((X_total,X_tmp))
        y_total = np.vstack((y_total,y_tmp))
        openvals_total = np.vstack((openvals_total,openvals_tmp))
        
#X_total, y_total, openvals_total = loaddata(dn,nb,yd)

In [5]:
print("X: {}, y: {}, ov: {}".format(X_total.shape, y_total.shape, openvals_total.shape))

X: (1796, 80), y: (1796, 1), ov: (1796, 1)


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu,linear
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split

In [7]:
#split the data using sklearn routine 
X_train, X_, y_train, y_ = train_test_split(X_total,y_total,test_size=0.3, random_state=1)
X_test, X_cv, y_test, y_cv = train_test_split(X_, y_, test_size=0.3, random_state=1)
print("X_train.shape", X_train.shape, "y_train.shape", y_train.shape)
print("X_test.shape", X_test.shape, "y_test.shape", y_test.shape)
print("X_cv.shape", X_cv.shape, "y_cv.shape", y_cv.shape)

X_train.shape (1257, 80) y_train.shape (1257, 1)
X_test.shape (377, 80) y_test.shape (377, 1)
X_cv.shape (162, 80) y_cv.shape (162, 1)


In [8]:
m,n = X_train.shape
print(m)
print(n)

1257
80


In [9]:
model = Sequential(
    [
        tf.keras.Input(shape=(n,)),
        Dense(units=16, activation='relu', name = 'layer1'),
        Dense(units=12, activation='relu', name = 'layer2'),
        Dense(units=10, activation='relu', name = 'layer3'),
        Dense(units=1, activation='linear', name= 'out')
    ]
)

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 16)                1296      
_________________________________________________________________
layer2 (Dense)               (None, 12)                204       
_________________________________________________________________
layer3 (Dense)               (None, 10)                130       
_________________________________________________________________
out (Dense)                  (None, 1)                 11        
Total params: 1,641
Trainable params: 1,641
Non-trainable params: 0
_________________________________________________________________


In [11]:
cost_fn = tf.keras.losses.MeanSquaredError()
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

model.fit(
    X_train,y_train,            
    epochs=100,
)

Train on 1257 samples
Epoch 1/100
1257/1257 [==============================] - 2s 1ms/sample - loss: 8.1597
Epoch 2/100
1257/1257 [==============================] - 0s 119us/sample - loss: 5.2837
Epoch 3/100
1257/1257 [==============================] - 0s 136us/sample - loss: 4.6741
Epoch 4/100
1257/1257 [==============================] - 0s 161us/sample - loss: 4.7134
Epoch 5/100
1257/1257 [==============================] - 0s 152us/sample - loss: 4.3420
Epoch 6/100
1257/1257 [==============================] - 0s 154us/sample - loss: 4.7019
Epoch 7/100
1257/1257 [==============================] - 0s 136us/sample - loss: 4.4869
Epoch 8/100
1257/1257 [==============================] - 0s 172us/sample - loss: 4.3458
Epoch 9/100
1257/1257 [==============================] - 0s 120us/sample - loss: 4.1046
Epoch 10/100
1257/1257 [==============================] - 0s 114us/sample - loss: 3.9797
Epoch 11/100
1257/1257 [==============================] - 0s 109us/sample - loss: 3.8968
Epoch 12/1

Epoch 93/100
1257/1257 [==============================] - 0s 115us/sample - loss: 2.8765
Epoch 94/100
1257/1257 [==============================] - 0s 121us/sample - loss: 2.7546
Epoch 95/100
1257/1257 [==============================] - 0s 118us/sample - loss: 2.8745
Epoch 96/100
1257/1257 [==============================] - 0s 132us/sample - loss: 2.6642
Epoch 97/100
1257/1257 [==============================] - 0s 140us/sample - loss: 2.6463
Epoch 98/100
1257/1257 [==============================] - 0s 115us/sample - loss: 2.5748
Epoch 99/100
1257/1257 [==============================] - 0s 114us/sample - loss: 2.5575
Epoch 100/100
1257/1257 [==============================] - 0s 119us/sample - loss: 2.7700


In [12]:
predictions = model.predict(X_test)

In [13]:
i=0
ngood = 0
for yval, pval in zip(y_test, predictions):
    isgood = yval[0]*pval[0] > 0
    i+=1
    if isgood:
        ngood += 1
    #print("y: {: .2f}  p: {: .2f} samedir: {}".format(yval[0],pval[0], isgood ))

accuracy = ngood/i
print("Accuracy: {}".format(accuracy))

Accuracy: 0.870026525198939
